In [1]:
from brainwidemap import bwm_query
import numpy as np
from one.api import ONE
from brainbox.population.decode import get_spike_counts_in_bins
from brainbox.io.one import SpikeSortingLoader, SessionLoader
from ibllib.atlas import AllenAtlas


Downloading: K:\Flatiron\ONE\alyx.internationalbrainlab.org\tmpnmp6m81p\cache.zip Bytes: 90313069


100%|████████████████████████████████████████████████████| 86.12925434112549/86.12925434112549 [01:52<00:00,  1.30s/it]


In [11]:
from brainwidemap import bwm_query
import numpy as np
from one.api import ONE
one = ONE() 
ba = AllenAtlas()

# load info about all released sessions
bwm_df=bwm_query(one=None, alignment_resolved=True, return_details=False, freeze='2022_10_bwm_release')

Loading bwm_query results from fixtures/2022_10_bwm_release.csv


In [10]:
bwm_df

,pid,eid,probe_name,session_number,date,subject,lab
0,56f2a378-78d2-4132-b3c8-8c1ba82be598,6713a4a7-faed-4df2-acab-ee4e63326f8d,probe00,1,2020-02-18,NYU-11,angelakilab
1,47be9ae4-290f-46ab-b047-952bc3a1a509,56956777-dca5-468c-87cb-78150432cc57,probe01,1,2020-02-21,NYU-11,angelakilab
2,6be21156-33b0-4f70-9a0f-65b3e3cd6d4a,56956777-dca5-468c-87cb-78150432cc57,probe00,1,2020-02-21,NYU-11,angelakilab
3,1e176f17-d00f-49bb-87ff-26d237b525f1,a8a8af78-16de-4841-ab07-fde4b5281a03,probe00,1,2020-01-22,NYU-12,angelakilab
4,701026df-e170-4ca7-88aa-eb0b95ef6ba1,a8a8af78-16de-4841-ab07-fde4b5281a03,probe01,1,2020-01-22,NYU-12,angelakilab
...,...,...,...,...,...,...,...
542,8bf0f1a4-0d8c-4df3-a99e-f7c81c809652,993c7024-0abc-4028-ad30-d397ad55b084,probe01,1,2020-09-16,CSH_ZAD_029,zadorlab
543,5d570bf6-a4c6-4bf1-a14b-2c878c84ef0e,fece187f-b47f-4870-a1d6-619afe942a7d,probe01,1,2020-09-17,CSH_ZAD_029,zadorlab
544,f7c93877-ec05-4091-a003-e69fae0f2fa8,fece187f-b47f-4870-a1d6-619afe942a7d,probe00,1,2020-09-17,CSH_ZAD_029,zadorlab
545,675952a4-e8b3-4e82-a179-cc970d5a8b01,c7bd79c9-c47e-4ea5-aea3-74dda991b48e,probe01,1,2020-09-19,CSH_ZAD_029,zadorlab


In [54]:
eid=bwm_df['eid'][0]

dict_keys(['stimOff_times', 'goCueTrigger_times', 'intervals_bpod', 'firstMovement_times', 'goCue_times', 'probabilityLeft', 'response_times', 'feedbackType', 'rewardVolume', 'contrastRight', 'choice', 'feedback_times', 'stimOn_times', 'contrastLeft', 'intervals'])

In [62]:
# 1. USE one.list_collections
# The format of the returned datasets gives the path of the collection followed by the dataset. 
# e.g in the case of alf/trials.table.pqt, alf is the collection and trials.table.pqt is the dataset. 
# The collection is important as it differentiates datasets with the same name e.g spikes.times 
# in alf/probe00/pykilosort and spikes.times in alf/probe01/pykilosort.

# The unique group of collections that belong to a session can be listed using the one.list_collections method,

collections = one.list_collections(eid)
print(collections)

eid='6713a4a7-faed-4df2-acab-ee4e63326f8d'
files = one.load_collection(eid, 'alf', download_only=True)

# Download all data in alf collection for specific probe
insertion = one.alyx.rest('insertions', 'list', session=eid)[0]
probe_label = insertion['name']
files = one.load_collection(eid, f'alf/{probe_label}/pykilosort', download_only=True)

['alf/probe01/pykilosort', 'alf/probe00', 'alf/probe00/pykilosort', 'alf/probe01', 'raw_ephys_data/probe00', 'raw_ephys_data/probe01', 'alf', 'raw_passive_data', 'raw_ephys_data', 'raw_behavior_data', 'spike_sorters/pykilosort/probe01', 'raw_video_data', 'spike_sorters/pykilosort/probe00', 'spike_sorters/ks2_matlab/probe00', 'spike_sorters/ks2_matlab/probe01']


In [ ]:
# 2. USE one.load_dataset
# A single dataset can be downloaded and loaded into memory by passing in 
# the eid and dataset as arguemnts into the one.load_dataset method,

# Download and load the left camera timestamps
left_cam_times = one.load_dataset(eid, '_ibl_leftCamera.times.npy')

# Download and load the spikes times for probe00
spike_times = one.load_dataset(eid, 'spikes.times.npy', collection='alf/probe00/pykilosort')

wheel_times = one.load_dataset(eid, '_ibl_wheel.timestamps.npy')

In [67]:
# 3. USE one.load_object
# A group of attributes (e.g amps, depths, metrics) belonging to the same object (e.g clusters) 
# can be downloaded and loaded in one command using the one.load_object method

# Load in all trials datasets
trials = one.load_object(eid, 'trials', collection='alf')
# Load wheel datasets
wheel = one.load_object(eid, 'wheel', collection='alf')

# Only download the clusters object for probe01
clusters = one.load_object(eid, 'clusters', collection=f'alf/{pname}/pykilosort', download_only=True)

# Only download the spikes object for probe01
spikes = one.load_object(eid, 'spikes', collection=f'alf/{pname}/pykilosort',  download_only=True)